In [11]:
from ultralytics import YOLO

# Load a model
model = YOLO('runs/detect/train2/weights/best.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model(['../imgs/card_0.png', '../imgs/card_18.png'])  # return a list of Results objects

# Process results list
for i, result in enumerate(results):
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.save(filename=f'result_{i}.jpg')  # save to disk



0: 640x640 (no detections), 32.7ms
1: 640x640 (no detections), 32.7ms
0: 640x640 (no detections), 32.7ms
1: 640x640 (no detections), 32.7ms
Speed: 2.5ms preprocess, 32.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
Speed: 2.5ms preprocess, 32.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


In [12]:

import cv2
import numpy as np

def find_arrow_direction(image_path):
    # Load the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Assume the largest contour is the arrow.
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    if contours:
        cnt = contours[0]

        # Approximate the contour to reduce the number of vertices
        epsilon = 0.01 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        # Find the bounding box and its center
        rect = cv2.minAreaRect(approx)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        center = np.mean(box, axis=0)

        # Find the tip of the arrow by identifying the vertex farthest from the center
        tip = max(approx, key=lambda x: np.linalg.norm(x-center))

        # Determine direction based on the tip's position relative to the center
        dx = tip[0][0] - center[0]
        dy = tip[0][1] - center[1]

        if abs(dx) > abs(dy):  # Horizontal arrow
            direction = "left" if dx < 0 else "right"
        else:  # Vertical arrow
            direction = "up" if dy < 0 else "down"

        return direction
    else:
        return "No arrow found"



In [13]:
image_path = "../imgs/card_18.png"
direction = find_arrow_direction(image_path)
print(f"Arrow direction: {direction}")

Arrow direction: up


/tmp/ipykernel_2946281/2413554232.py:25: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


In [14]:
image_path = "../imgs/card_14.png"
direction = find_arrow_direction(image_path)
print(f"Arrow direction: {direction}")

Arrow direction: down


/tmp/ipykernel_2946281/2413554232.py:25: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


In [15]:
image_path = "../imgs/card_13.png"
direction = find_arrow_direction(image_path)
print(f"Arrow direction: {direction}")

Arrow direction: left


/tmp/ipykernel_2946281/2413554232.py:25: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


In [16]:
import cv2
import numpy as np

def recognize_rectangles(image_path):
    # Load the image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    rectangles = []
    for cnt in contours:
        # Approximate the contour
        epsilon = 0.05 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        # Check if the approximated contour has 4 vertices (possible rectangle)
        if len(approx) == 4:
            # Optional: Check aspect ratio to filter out non-rectangular shapes
            # rect = cv2.minAreaRect(cnt)
            # width, height = rect[1]
            # if 0.8 < width/height < 1.25:
            rectangles.append(cnt)

    # Draw detected rectangles on the original image
    cv2.drawContours(img, rectangles, -1, (0, 255, 0), 3)

    return img
    # Show the output
    #cv2.imshow('Detected Rectangles', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# Example usage
image_path = '../imgs/card_14.png'
recognize_rectangles(image_path)

array([[[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0, 255,   0],
        ...,
        [  0, 255,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]